In [ ]:
import os, sys
import numpy as np 
import gmat_nav as gnav
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
# Directory location of this Jupyter notebook tutorial
file_dir = os.path.abspath('') #os.path.dirname(os.path.abspath(__file__))
# --------------- USER DEFINED PATHS (USER -- PLEASE LOOK HERE!) ------------------# 
gmat_root_dir = '/home/natsubuntu/Desktop/SysControl/estimation/CauchyCPU/CauchyEst_Nat/GMAT'
gmat_data_dir = file_dir + "/data"
eop_filepath = gmat_data_dir + "/eop_file.txt"
spaceweather_filepath = gmat_data_dir + "/SpaceWeather-v1.2.txt"
gnav.check_then_load_gmatpy(gmat_root_dir, eop_filepath, spaceweather_filepath)
gmat = gnav.gmat # our handle to the gmat module, loaded through gnav
gmat.Clear() # clears any junk lying around 

# Our Satellite Parameters
# Remember, your intial epoch time must be UTC Gregorian, otherwise use time_convert(...) function in gnav to switch it!
t0 = datetime(year=2023, month=7, day=10, hour=19, minute=34, second=54, microsecond=0)
pos3 = np.array([4.9962452882705193e+03,3.8779464630861030e+03,2.7360432364171807e+03]) # approx a 550 (units -> kilometer) orbit
vel3 = np.array([-5.0280935744461930e+00,5.5759213419992673e+00,1.2698611722905329e+00]) # speed (units -> kilometers/second) 
x0 = np.concatenate((pos3, vel3)) # Your initial state vector
Cd0 = 2.1 # nominal coefficient of drag # unitless
Cr0 = 0.75 # nominal coefficient of reflectivity # unitless
A = 14.18 # (Drag) Area of the satellite # units -> meters
m = 3995.6 # (dry) mass of the satellite # units -> kilograms
fuel_mass = 359.9 # units -> kilograms
solar_flux = 1370.052 # units -> Watts/m^2
dt = 60.0 # units -> seconds (Our Time Step)

Welcome to tutorial three! Here we introduce how to use the GMAT Python API to construct components needed by an estimation algorithm, such as an extended Kalman filter. Such components include:

> 1.) Constructing the Jacobian of your orbiter

<br>

> 2.) Constructing the State Transition Matrix of your orbiter, for a time increment of $t_k$ to $t_{k+1}$ (of your choosing). This process uses the Jacobian.

<br>

> 3.) Transforming between coordinate frames. Specifically, transforming the orbiter's position vector from the EarthMJ2000Eq coordinate frame to the earth body fixed coordinate frame. We will have to do so as GPS measurements are typically provided in the earth body fixed frame, whereas the orbiter uses the EarthMJ2000Eq coordinate frame.

<br>

> 4.) Adding 'solve for' states to our estimation problem. Solve for states help estimate changes to modeling parameters, such as our nominal coefficient of drag or the nominal coefficient of reflectivity. Solve for states, as the name implies, are additional states we add explicitly to the state vector. Pedantically, our measurements (here, GPS) are used to estimate the solve for states.

<br>
Lets get started!

Using GMAT, constructing the Jacobian of the state vector (for now, our three positions and three velocities) using GMAT is simple. The object returned by gnav.EarthOrbitingSatellite() has methods to do this for you. Following the spirit of these tutorials, however, we will first show the exact sequence of GMAT calls used, and then introduce the relevant method which does this for you.

The following shows you how to access the Jacobian, bootstrapping off our EarthOrbitingSatellite class already introduced. It also shows how to access time derivatives of the state.

In [ ]:
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)

# Get Time Derivatives and Jacobian
pstate = sat.gator.GetState() 
sat.fm.GetDerivatives(pstate, dt=dt, order=1)
fdot = sat.fm.GetDerivativeArray()
dx_dt = np.array(fdot[0:6])
Jac = np.array(fdot[6:42]).reshape((6,6))
print("dx/dt:\n", dx_dt) # 6 x 1 vector
print("Jacobian:\n", Jac) # 6 x 6 matrix

The above requires handoffing off the current system state (retrieved through the propagator) to the force model. Calling GetDerivativeArray() returns a vector of length 42, where the first 6 elements are the time derivatives of the state vector, while the next 36 elements are the Jacobian elements. Reshaping the Jacobian elements into a 6x6 matrix yields the intended result. Note that in the GetDerivatives(...) call you should specify your time step (the 'order' must be set to 1).

A VERY important mention is that within create_model(...), the Propagation State Manager's SetProperty("AMatrix") method is called. If this is not done, calling fm.GetDerivativeArray() will return only the time derivatives of the state vector, and not the Jacobian information. You must explicitly tell GMAT that Jacobian information should be returned. Take a look at gnav.EarthOrbitingSatellite.create_model() to see this for yourself. 

One can accomplish the above using the built in sat.get_jacobian_matrix() method. This will return the jacobian of the satellite for its current state. The following shows this matches the latter.

In [ ]:
print("Jacobian:\n", sat.get_jacobian_matrix())

Step one complete. Onto step two.

The state transition matrix (STM) can be computed in a variety of ways. An easy solution, for relatively small time steps, is to use a power series approximation. This approximation gets better and better for smaller time increments. Mathematically, a simple approximation to the STM for a nonlinear continous time dynamic system $\frac{dx}{dt} = f(x_{t_k})$ with time step $\Delta = t_{k+1}-t_k$ can be found as

\begin{equation}
\Phi_k(t_{k+1},t_k) = \sum_{i=0}^{L}\frac{\nabla_x{f(x_{t_k})}^i\Delta^i}{i!},
\end{equation}
where $L$ is the power series order, $\nabla_x{f(x_{t_k})}$ is the Jacobian (gradient taken with numerator derivative layout), and $\nabla_x{f(x_{t_k})}^i$ is the $i$-th matrix power of $\nabla_x{f(x_{t_k})}$. For example, given that $A$ is a square matrix, $A^3 = A\cdot A \cdot A$.  Using the jacobian we've already computed, this is a cinch. 

You may be wondering why, similar to obtaining the Jacobian, we don't just ask GMAT for the STM. Unfortunately, GMAT's API does not provide this feature out-of-the-box. That does not mean we cannot get an extremely close approximation to what its (internal) extended Kalman filter computes. The following matches the STMs computed by GMAT very closely. Skip the math blurb below if uninterested.  


Two improvements make our STM much better. First, computing the averaged jacobian $\bar{J} = \frac{\nabla_x{f(x_{t_{k+1}})} + \nabla_x{f(x_{t_k})}}{2}$ will result in a better approximation to the Jacobian over the time increment $\Delta$. To do so, we would need to first find the jacobian at $x_{t_{k}}$, invoke the propagator to get to $x_{t_{k+1}}$, then find the jacobian at $x_{t_{k+1}}$. This may be good enough for a small time increment $\Delta$. Second, smaller time steps yield a better STM approximation, as was alluded to before. One can create a much better STM approximation by computing a sequence of STMs for substeps $\Delta_s < \Delta$ and multiplying the results together. Let $\{x_{t_k}, x_{t_{k+\frac{1}{s}}}, x_{t_{k+\frac{2}{s}}}, ..., x_{t_{k+1}}\}$ refer to the trajectory of states between $x_{t_k}$ and $x_{t_{k+1}}$, with $s$ the number of substeps taken. The propagator will yield this sequence if we use a time increment $\Delta_s = \frac{\Delta}{s}$, rather than $\Delta$.


Mathematically, putting these ideas together translates as 

\begin{align}
    \Phi_k(t_{k+1},t_k) &= \prod_{j=0}^{s-1} \Phi_j(\Delta_s)\\
    \Phi_j(\Delta_s) &= \sum_{i=0}^{L}\frac{\left(\bar{J}_j\right)^i\Delta_s^i}{i!},\\
    \bar{J}_j &= \frac{\nabla_x{f(x_{t_{k+\frac{j+1}{s}}})} + \nabla_x{f(x_{t_k+\frac{j}{s}})}}{2},
\end{align}
where $\prod$ denotes the matrix multiplication product with ordering $\bar{\Phi}(\Delta_s)_{s-1} \bar{\Phi}(\Delta_s)_{s-2}, \dots \bar{\Phi}(\Delta_s)_{0}$ for the multiplication.

This is exactly the idea(s) used in the function sat.get_transition_matrix(...), which can be configured with three arguments: the power series order, the number of substeps, and whether to average the jacobian (possibly over each substep, if using substeps)

In [ ]:
# First, use no sub-steps or averaging, which will simply compute the STM with the jacobian at x_t_k
power_order = 3
num_sub_steps = None # could also set to 1
with_avg_jacobian = False
Phi = sat.get_transition_matrix(power_order, num_sub_steps, with_avg_jacobian)
print("STM using Non-Averaged Jacobian and No Substeps:\n",Phi)

# Now, use sub-steps, which will compute a better STM by using the product of STMs over the sub_step's time_increment
num_sub_steps = 6 # Each sub-step corresponds to 10 seconds of propagation
with_avg_jacobian = False
Phi_substeps = sat.get_transition_matrix(power_order, num_sub_steps, with_avg_jacobian)
print("STM using Non-Averaged Jacobian and Substeps:\n", Phi_substeps)

# Now, use sub-steps and the averaged jacobian of each substep, which will compute the best STM approximation
num_sub_steps = 6 # Each sub-step corresponds to 10 seconds of propagation
with_avg_jacobian = True
Phi_substeps_avg = sat.get_transition_matrix(power_order, num_sub_steps, with_avg_jacobian)
print("STM using Average Jacobian and Substeps:\n", Phi_substeps_avg)

We can see the results are close, but not exact. Somewhat obviously, calling get_transition_matrix(...) without substeps and without averaging will be fastest, but less accurate. It was seen that the function call above using sub-steps of approximately 20 seconds or less and averaging delivers an STM that differs by no more than $\approx0.5\%$ from that of GMAT's baked-in EKF. This was seen when comparing the function above to STMs computed internally by GMAT's EKF and then logged to disk. Using sub-steps and averaging, however, will induce a bit of overhead.

Onto Step 3:

Transforming between coordinate frames will become important when working with GPS data, which is typically logged in the Earth Body Fixed frame. The satellite however, uses the EarthMJ2000Eq frame. In a Kalman filter, we will need to transform our state into what we would expect the measurement to be. To do this, we will need to create a Coordinate Converter object, as well as the the individual coordinate frames. The following shows how to convert a state from the inertial EarthMJ2000Eq frame into the EarthBodyFixed frame.

In [ ]:
# Clear Satellite Model to allow this standalone example to run
sat.clear_model()

# Before Initialization, make two coordinate converters to help convert MJ2000Eq Frame to ECF Frame
ecf = gmat.Construct("CoordinateSystem","ECF","Earth","BodyFixed") # Body Fixed Earth Frame
eci = gmat.Construct("CoordinateSystem","ECI","Earth","MJ2000Eq") # Inertial Earth Frame
csConverter = gmat.CoordinateConverter()
# Perform top level initialization
gmat.Initialize()

# Current time in UTC Gregorian
t_gps_example = t0 #gnav.time_string_2_datetime(t0) #+ timedelta(seconds = 60)
x_gps_example = x0.copy() 

# Convert time to A1 MJD 
time_a1mjd = gnav.time_convert(t_gps_example, "UTC", "A1")["out_mjd"]
# GMAT requires the state vector inputs to be type Rvector6
state_in = gmat.Rvector6(*list(x_gps_example)) # State in MJ2000Eq
state_out = gmat.Rvector6() # State in Earth Body Fixed Coordinates
csConverter.Convert(time_a1mjd, state_in, eci, state_out, ecf)
ecf_array = np.array([ state_out[0], state_out[1], state_out[2], state_out[3], state_out[4], state_out[5] ])
print("State Earth Inertial:\n", x_gps_example)
print("State Earth Body Fixed:\n", ecf_array)

The procedure is somewhat intuitive: Create both coordinate frames of interest, and the CoordinateConverter object. Then, convert the current time to A1 MJD (the time_convert(...) function is very helpful for this). Use the Rvector6 class to pass the state info in and out (I'm converting the output back to a numpy array after), and voila. 

Two functions have been implemented for your convienience to work with coordinate conversions, as part of our EarthOrbitingSatellite object.

> sat.transform_Earth_MJ2000Eq_2_BodyFixed(state_mj2000 = None, time_a1mjd = None) -> Transforms the state given in MJD2000Eq frame into EarthBodyFixed frame. Since the satellite is setup for the Earth's MJD2000Eq frame, no arguments are truly needed; the function will work plain and simple for the current state and time (epoch) of the satellite in question. If you wish to transform a state, which differs from that of the current satellite state, provide it to the 'state_mj2000' argument. If nothing is provided, the function defaults to using the current satellite state. In the same sense, if the time also varies from the current satellite epoch, provide the time to 'time_a1mjd' (use the gnav.time_convert(...) function to help out and get your time into A1 MJD). Therefore, this function can be used easily for the Satellite itself, or for a general coordinate conversion of your choosing.

<br> 

> sat.transform_Earth_BodyFixed_2_MJ2000Eq(self, state_earth_bf, time_a1mjd = None) -> Transforms the state given in the EarthBodyFixed frame to the MJD2000Eq frame. This function may be helpful to turn the GPS point solutions one would use as measurements into the MJD2000Eq frame. Here, you have to provide at least the state. The function defaults to the current epoch, but again, you can provide any epoch you wish.

Both functions also work if your state input is only the position vector (a vector of length 3 and not 6). It will simply append a trivial velocity of zero to your input, in the intermediate calculations.

Lets see how this works below:

In [ ]:
sat.clear_model()
# Recreate the model, as it was deleted above in the previous cell block
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)
print("x0_eci:\n", sat.get_state() )
x_ebf = sat.transform_Earth_MJ2000Eq_2_BodyFixed() # inertial 2 body frame
print("x0_ebf:\n", x_ebf)
x_eci = sat.transform_Earth_BodyFixed_2_MJ2000Eq(x_ebf) # body 2 inertial frame
print("x0_eci:\n", x_eci)
x_ebf[3:] *= 0
print("x0_ebf (stationary):\n", x_ebf) # stationary point in body frame
x_eci = sat.transform_Earth_BodyFixed_2_MJ2000Eq(x_ebf[0:3]) # or call on whole vector and not just poistion
print("x0_eci (velocity is that of earth):\n", x_eci)

# For an example, show after a propagation too
sat.step()
print("\nTime Stepping!\n\nx1_eci:\n", sat.get_state() )
x_ebf = sat.transform_Earth_MJ2000Eq_2_BodyFixed() # inertial 2 body frame
print("x1_ebf:\n", x_ebf)
x_eci = sat.transform_Earth_BodyFixed_2_MJ2000Eq(x_ebf) # body 2 inertial frame
print("x1_eci:\n", x_eci)


It will also be very useful to retrive the exact rotation matrix that is used to convert one frame's position to another: For example in the transformation
\begin{equation}
    r_b = H r_i, \quad r_b, r_i \in \mathbb{R}^3
\end{equation}
the position vector $r_i$ is transformed to position $r_b$ through the rotation matrix H. When modelling GPS measurements, the position of the orbiter in the interial frame can be converted to the body frame via
\begin{equation}
    \bar{z}_k = [H, 0] x_k
\end{equation}
where H is a rotation matrix, like the one above. After calling a coordinate conversion, we can retrieve the rotation matrix H (which defines the frame transformation previously specified) as:

In [ ]:
x2_eci = sat.step()
print("x2_eci:\n", x2_eci)
x2_ebf = sat.transform_Earth_MJ2000Eq_2_BodyFixed()
print("x2_ebf:\n", x2_ebf)
Hpos = sat.get_last_position_rotation_matrix()
print("(Position) x2_ebf[0:3] => Hpos @ x2_eci\n", Hpos @ x2_eci)
print("Hpos:\n", Hpos[0:3,0:3])

The above shows that this coordinate conversion is nothing more than a matrix vector multiplication, as suggested previously. Note that you must call sat.transform_Earth_MJ2000Eq_2_BodyFixed(...)
or sat.transform_Earth_BodyFixed_2_MJ2000Eq(...) before using sat.get_last_position_rotation_matrix(), as it retrives the rotation matrix for the LAST!! coordinate transformation made.

 Unfortunately, it appears as though GMAT's API is not setup to return the rotation matrix which converts velocities between frames. While such functionality will likely not be used as frequently, a numerical differentation scheme could back-out this transformation by auto-differentiating the relevant "sat.transform_" function. This is left to future work, but is very trivial.

Onto step 4:

Last thing we need is the ability to include 'solve for' states in our estimation pipeline. Solve for states allow us to adaptively update model coefficients, such as $C_d$ and $C_r$ (coefficient of drag, coefficient of reflectivity) by modelling changes to these values as states. For example, the drag coefficient could be replaced as $C_d = C_{d0}(1 + \frac{dC_d}{C_{d0}})$ where $C_{d0}$ is our nominal guess for the coefficient of drag, $\frac{dC_d}{C_{d0}}$ is our $\textit{solve for state}$ (change in coefficient of drag) and
\begin{align}
    \frac{d\left(\frac{dC_d}{C_{d0}}\right)}{dt} = \frac{-1}{\tau}\cdot\left(\frac{dC_d}{C_{d0}}\right) + w
\end{align}
is (an assumed) first order Gauss Markov (FOGM) model for the solve for state. Above, $\tau$ is the FOGM modelling parameter and $w$ is (Gaussian) process noise. Choosing $\tau$ and a value for the PSD of $w$ can be accomplished by analyzing space weather data for the relevant parameter. Including solve for states makes an estimator more robust to modelling errors or sudden anomoly, as the estimator can update its value to account for residual errors that persist.

The EarthOrbitingSatellite class allows for the inclusion of solve for states. Specifically, $C_d$ and $C_r$ are already baked in. However, one can modify this API to include others. 

A notable mention is that in the analysis of space weather data, the distribution that best describes changes to these solve for states is sometimes not Gaussian. Sometimes, far from it. For example, changes in atmospheric density can be $\textit{heavy tailed}$, meaning that it can undergo large and sudden changes at somewhat infrequent intervals. The Gaussian distribution does not model sudden, large and rare changes very well. This implies that it may be wise to simulate the process noise $w$ above as something that is non-Gaussian. Specifically, the (family of) distribution that better describe changes to atmospheric density (somewhat equivalently, coefficient of drag) or coeffifient of reflectivity, are called symmetric alpha-stable ($S-\alpha-S$) distributions. $S-\alpha-S$ distributions are a general class of distribution described by a characteristic function (CF), essentially, the fourier transformation of a probability density function (pdf), as 
\begin{equation}
    \phi_X(\nu) = \exp(-\sigma|\nu|^\alpha + jb\nu)
\end{equation}
where above, $\phi_X(\nu)$ denotes the CF with $\nu\in\mathbb{R}$ the spectral variable, $\sigma>0$ is a scaling parameter describing the "width" of the distribution (think of standard deviation), $\alpha \in (0,2]$ classifies how rare "heavy-tailed" events are, and $b\in\mathbb{R}$ is the location parameter describing the value for which the distribution is centered about. Typically $b=0$, as noise distributions are usually zero-centered. When $\alpha=2$, the inverse CF yields the Gaussian pdf, whereas setting $\alpha=1$ yields the Cauchy distribution. We can think of the spectrum of $\alpha$ as a way to classify how likely the occurence of a rare, outlier event is. As $\alpha$ is lowered from $2$ (Gaussian) to $1$ (Cauchy), the occurence of rare events have much more likelihood, as the amount of mass in the tail of the pdf increases. This is a better definition of "heavy-tailed" (i.e, $\alpha<2$). 

For $\alpha\in(1,2)$, the closed form pdf for the inverse to the CF above does not exist. However, we can certainly still sample from distribution with $1 < \alpha < 2$, using methods from literature. The EarthOrbitingSatellite class allows for the inclusion of solve for states. Specifically, $C_d$ and $C_r$ are already baked in. The class allows modelling of solve for states using a first order markov (FOM) model. Additionally, we can set the noise distribution to anything with $\alpha\in[1,2]$, meaning we can simulate Gaussian noises (set $\alpha=2$) and all the way down to Cauchy noise (set $\alpha=1$). Generally, the distribution of space weather parameters are best modelled by something in between Gaussian and Cauchy, for example, $\alpha=1.5$. Of course, this requires careful analysis.

First, we'll show how to add solve for states that are Gaussian forced. Then well do the same for solve-fors that follow a $S-\alpha-S$ distribution for $\alpha < 2$ (Non-Gaussian). 

In [ ]:
# Add solve fors and call simulate -- First case: solve for with Gaussian noise
sat.clear_model()
# Recreate the model -- with Cd0 our "nominal" Cd value
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)
# Add a solve for state for Cd, as a FOGM model
sf_scale = 0.0013 # in Gaussian noise, this is 'standard deviation'
sf_tau = 21600 # half life of 6 hours
sat.set_solve_for(field = "Cd", dist="gauss", scale = sf_scale, tau = sf_tau) # Adds a seventh state

# Simulate satellite trajectory
num_steps = 500 # number of simulation steps
gps_std_dev = 0.0075 # kilometers, for each of the three (x,y,z) direction...Truly, this is not used right now
xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=None) # returns states, simulated gps measurements, process noise samples for the realization, and measurement noise samples from the realization.

Ts = np.arange(num_steps+1) * dt / 3600 # -> hours
plt.plot(Ts, Cd0*(1+xs[:,6]) ) # plot simulated Cd values, transforming change in Cd values to the time history of Cd values themselves
plt.xlabel("Time (hours)")
plt.ylabel("Coefficient of Drag")
plt.show()

Neat, right? 

Now is also a good time too to comment on the simulate(...) function above. It runs a full dynamic simulation, returning the states, simulated gps measurements, process noise samples from the realization, and measurement noise samples from the realization. In a simulation enviroment, this function provides the measurements a Kalman filter would process, but also returns the true simulation states, as well as the simulated process/measurement noises, respectively. The GPS sensor's noise standard deviation must be specified for the measurements, which is assumed Gaussian. Assigning a std. dev. of zero returns a perfect measurement and of course no corruptive noise. Additive process noise is only added to position and velocity during the simulation if you assign a covariance matrix to argument 'W' in the function call. Otherwise, no additive position/veloicty process noise is simulated. Additionally, the simulated GPS point solutions (the measurements) are returned in the earth body fixed frame.

Each time "step()" is internally called inside simulate(..), both the satellite position/velocity updates, along with each added solve for state. Moreover, the solve for states are given a "noisy" propagation in simulate(...). Meaning, a sample is drawn from your solve-for's assigned noise distribution, and is added via the FO(G)M model suggested above. This of course then affects your simulated position and velocity state trajectories; as after the solve for is propagated, the new (noisy) nominal value is re-assigned to GMAT via a SetField call (see source code, its not terribly complicated). Now, we have a way to simulate atmosphere changes, and see the dynamic affect on position/velocity. 

Lets see how assigning an alpha stable noise distribution to our solve-for-state Cd changes things:

In [ ]:
# Add solve fors and call simulate -- Second case: solve for with alpha stable noise
sat.clear_model()
# Recreate the model -- with Cd0 our "nominal" Cd value
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt)
# Add a solve for state for Cd, as a FOGM model
sf_scale = 0.0013 # scaling parameter for the S-a-S distribution characterizing changes in Cd
sf_tau = 21600 # half life of 6 hours
sf_alpha = 1.5 # alpha value of SaS distrbution -> between Cauchy and Gaussian
sat.set_solve_for(field = "Cd", dist="sas", scale = sf_scale, tau = sf_tau, alpha = sf_alpha) # Adds a seventh state

# Simulate satellite trajectory
num_steps = 500 # number of simulation steps
gps_std_dev = 0.0075 # kilometers, for each of the three (x,y,z) direction...Truly, this is not used right now
xs, zs, ws, vs = sat.simulate(num_steps, gps_std_dev, W=None) # returns states, simulated gps measurements, process noise samples for the realization, and measurement noise samples from the realization.

Ts = np.arange(num_steps+1) * dt / 3600 # -> hours
plt.plot(Ts, Cd0*(1+xs[:,6]) ) # plot simulated Cd values, transforming change in Cd values to the time history of Cd values themselves
plt.xlabel("Time (hours)")
plt.ylabel("Coefficient of Drag")
plt.show()

If you run the above cell block several times, you will see that the Coefficient of Drag sometimes has large impulsive changes. This is consistent with space weather data analysis for changes in atmospheric density (equivalently a modeled solve for state on Cd), coefficient of reflectivity, and others. Lowering the alpha-value even further results in more frequent and larger impulses. 

A notable mention is that the EarthOrbitingSatellite() Class will return Jacobians/STMs that includes your solve for states. That is, if you have two solve for states, calling sat.get_jacobian_matrix() or sat.get_transition_matrix() will return the Jacobian or STM of size (6+2) by (6+2), respectively. These are all filled out and ready to go. Take a look at the source code (get_jacobian_matrix) to see how the partial derivative information for solve for states are created. If you wish to add on top of this library, following the source code for the already existing implementations of $C_d$ and $C_r$ is the way to go.

We are all ready to put the ideas of tutorials 1 to 3 together and build an EKF. Head over to tutorial 4 to see it in action!